In [1]:
from math import log
import operator


def testDataset():
    dataSet = [
        [0, 0, 0, 0, 'no'],
        [0, 0, 0, 1, 'no'],
        [0, 1, 0, 1, 'yes'],
        [0, 1, 1, 0, 'yes'],
        [0, 0, 0, 0, 'no'],
        [1, 0, 0, 0, 'no'],
        [1, 0, 0, 1, 'no'],
        [1, 1, 1, 1, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [2, 0, 1, 2, 'yes'],
        [2, 0, 1, 1, 'yes'],
        [2, 1, 0, 1, 'yes'],
        [2, 1, 0, 2, 'yes'],
        [2, 0, 0, 0, 'no']
    ]
    labels = ['年龄', '有工作', '有房', '信贷情况']  
    return dataSet, labels  


def splitDataset(dataSet, axis, value):
    retDataSet = []  
    for featVec in dataSet:  
        if featVec[axis] == value:  
            reducedFeatVec = featVec[:axis]  
            reducedFeatVec.extend(featVec[axis + 1:])  
            retDataSet.append(reducedFeatVec)  
    
    return retDataSet


def calcShannonent(dataSet):
    numEntires = len(dataSet)  
    labelCounts = {}  
    for featVec in dataSet:  
        currentLabel = featVec[-1]  
        
        if currentLabel not in labelCounts.keys():  
            labelCounts[currentLabel] = 0  
        labelCounts[currentLabel] += 1  
    shannonEntropy = 0.0  
    for key in labelCounts:  
        prob = float(labelCounts[key]) / numEntires  
        shannonEntropy -= prob * log(prob, 2)  
    return shannonEntropy


def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1  
    baseEntropy = calcShannonent(dataSet)  
    bestInfoGain = 0.0  
    bestFeature = -1  
    for i in range(numFeatures):  
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)  
        newEntropy = 0.0  
        for value in uniqueVals:  
            subDataSet = splitDataset(dataSet, i, value)  
            prob = len(subDataSet) / float(len(dataSet))  
            newEntropy += prob * calcShannonent(subDataSet)  
        infoGain = baseEntropy - newEntropy  
        
        if bestInfoGain < infoGain:
            bestInfoGain = infoGain  
            bestFeature = i  
    return bestFeature, bestInfoGain


def mostTags(classList):
    classCount = {}
    for vote in classList:  
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)  
    return sortedClassCount[0][0]  


def createTree(dataSet, labels, threshold_value=0.0001):
    classList = [example[-1] for example in dataSet]  
    if classList.count(classList[0]) == len(classList):  
        return classList[0]     
    if len(dataSet[0]) == 2:  
        return mostTags(classList)   
    bestFeat, currentInfoGain = chooseBestFeatureToSplit(dataSet)  
    if currentInfoGain < threshold_value:  
        return mostTags(classList)
    bestFeatLabel = labels[bestFeat] 
    myTree = {bestFeatLabel: {}} 
    del (labels[bestFeat])  
    featValues = [example[bestFeat] for example in dataSet]  
    uniqueVals = set(featValues)  
    for value in uniqueVals:  
        split_dataSet = splitDataset(dataSet, bestFeat, value)
        myTree[bestFeatLabel][value] = createTree(split_dataSet, labels)
    return myTree  


def classify(inputTree, testLabels, testVec):
    node_value = next(iter(inputTree))
    featIndex = testLabels.index(node_value)
    testVec_value = testVec[featIndex]
    sub_node = inputTree[node_value]
    result = sub_node.get(testVec_value, -1)
    if result != -1:
        if type(result).__name__ == 'dict':
            classLabel = classify(result, testLabels, testVec)
        else:
            classLabel = result
    else:
        classLabel = "nokey"
    return classLabel


if __name__ == '__main__':
    train_dataSet, train_labels = testDataset()
    myTree = createTree(train_dataSet, train_labels)
    print(myTree)

{'有房': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


In [2]:
def classify(inputTree, testLabels, testVec):
    # 获取tree的节点的value
    node_value = next(iter(inputTree))
    # 获取该值对应的分裂标签的索引下标
    featIndex = testLabels.index(node_value)
    testVec_value = testVec[featIndex]
    # 获取指定value的tree的node
    sub_node = inputTree[node_value]
    result = sub_node.get(testVec_value, -1)
    if result != -1:
        if type(result).__name__ == 'dict':
            classLabel = classify(result, testLabels, testVec)
        else:
            classLabel = result
    else:
        classLabel = "nokey"
    return classLabel

In [4]:
if __name__ == '__main__':
    # info()
    # 创建训练样本数据集及其对应的标签
    train_dataSet, train_labels = testDataset()
    s = train_dataSet[-2]
    # 保存决策树非叶节点的名称值(value)，即
    featLabels = []
    # 创建决策树
    # myTree = createTree(train_dataSet, train_labels, featLabels)
    myTree = createTree(train_dataSet, train_labels)

    # 输出决策树
    print(myTree)

    # 使用新的样本向量进行测试
    test_vec = [0, 0, 1, 0]
    test_labels = ['年龄', '有工作', '有房', '信贷情况']
    result = classify(myTree, test_labels, test_vec)
    print(result)

{'有房': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
yes


In [6]:

test_dataSet, test_labels = testDataset()
for test_vec in  test_dataSet:
    result = classify(myTree, test_labels, test_vec)
    if result == 'yes':
        print(result, "同意贷款")
    if result == 'no':
        print(result, "拒绝贷款")
    if result == 'nokey':
        print(result, "数据集中特征值错误")

no 拒绝贷款
no 拒绝贷款
yes 同意贷款
yes 同意贷款
no 拒绝贷款
no 拒绝贷款
no 拒绝贷款
yes 同意贷款
yes 同意贷款
yes 同意贷款
yes 同意贷款
yes 同意贷款
yes 同意贷款
yes 同意贷款
no 拒绝贷款
